In [49]:
import numpy as np
import theano
import string
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
import os
from os import path

In [50]:
import theano.tensor as T

In [51]:
def init_weight(Wi,Wo):
    return np.random.randn(Wi,Wo)/ np.sqrt(Wi+Wo)
def get_robert_frost():
    word2idx = {"START":0,"END":1}
    sentences = []
    current = 2
    with open('RobertFrost') as f:
        for line in f:
            line  = line.strip()
            if line:
                sentence = []
                line = line.translate(None, string.punctuation)
                for word in line.split():
                    if word not in word2idx:
                        word2idx[word] = current
                        current = current + 1
                    sentence.append(word2idx[word])
                sentences.append(sentence)
    return sentences,word2idx

In [52]:
sentences,vocabulary = get_robert_frost()

In [101]:
class lstm():
    def __init__(self,H,D,V):
        # number of hidden units
        self.H = H
        #dimension of the word embedding model
        self.D = D
        #vocabulary size model
        self.V = V
    def layer_init_weight(self):
        Wlx_init = init_weight(self.D,self.H)
        Wlh_init = init_weight(self.H,self.H)
        blh_init = np.zeros(self.H)
        
        thWlx = theano.shared(Wlx_init)
        thWlh = theano.shared(Wlh_init)
        thblh = theano.shared(blh_init)
        
        return thWlx,thWlh,thblh
    def set_lstm_variables(self):
        
        self.C0 = theano.shared(np.zeros(self.H))
        self.h0 = theano.shared(np.zeros(self.H))
        
        self.Wfx,self.Wfh,self.bfh = self.layer_init_weight()
        self.Wix,self.Wih,self.bih = self.layer_init_weight()
        self.WCx,self.WCh,self.bCh = self.layer_init_weight()
        self.Wox,self.Woh,self.boh = self.layer_init_weight()
    
    def set_variables(self):
        
        thX = T.ivector('thX')
        thY = T.ivector('thY')
        Ei = self.We[thX]
        
        
        #will set all lstm variable in instance variable params
        #will also set dparams variable
        self.params = [self.We,self.C0,self.h0,self.Wfx,self.Wfh,self.bfh,self.Wix,self.Wih,self.bih,
                      self.WCx,self.WCh,self.bCh,self.Wox,self.Woh,self.boh,self.Wo,self.bo]
        
        def recurrance(x_t,h_t1,C_t1):
            ft = T.nnet.sigmoid(x_t.dot(self.Wfx)+ h_t1.dot(self.Wfh) + self.bfh)
            it = T.nnet.sigmoid(x_t.dot(self.Wix)+ h_t1.dot(self.Wih) + self.bih)
            Cdasht = T.tanh(x_t.dot(self.WCx)+ h_t1.dot(self.WCh) + self.bCh)
            ot = T.nnet.sigmoid(x_t.dot(self.Wox)+ h_t1.dot(self.Woh) + self.boh)
            C_t = ft*C_t1 + it*Cdasht
            ot = T.nnet.sigmoid(x_t.dot(self.Wox)+ h_t1.dot(self.Woh) + self.boh)
            h_t = ot*T.tanh(C_t)
            return h_t,C_t
        
        #forward run of sequence
        [h,C],_ = theano.scan(
                                fn = recurrance,
                                sequences=Ei,
                                outputs_info=[self.h0,self.C0],
                                n_steps=Ei.shape[0]
                            )
        
        py_x = T.nnet.softmax(h.dot(self.Wo) + self.bo)
        
        prediction = T.argmax(py_x,axis=1)
        
        self.predict = theano.function(
            inputs = [thX],
            outputs = [prediction]
        )
    
    def fit(self,X,alpha=10e-5,mu=0.99,print_period=10,epochs=1):
        
        We_init = init_weight(self.V,self.D)
        self.We = theano.shared(We_init)
        
        thX = T.ivector('thX')
        thY = T.ivector('thY')
        Ei = self.We[thX]
        
        
        self.set_lstm_variables()
        ##set up output variables
        self.Wo = theano.shared(init_weight(self.H,self.V))
        self.bo = theano.shared(np.zeros(self.V))
        
        #will set all lstm variable in instance variable params
        #will also set dparams variable
        self.params = [self.We,self.C0,self.h0,self.Wfx,self.Wfh,self.bfh,self.Wix,self.Wih,self.bih,
                      self.WCx,self.WCh,self.bCh,self.Wox,self.Woh,self.boh,self.Wo,self.bo]
        #delta of the params
        self.dparams = [theano.shared(param.get_value()*0) for param in self.params]
        
        def recurrance(x_t,h_t1,C_t1):
            ft = T.nnet.sigmoid(x_t.dot(self.Wfx)+ h_t1.dot(self.Wfh) + self.bfh)
            it = T.nnet.sigmoid(x_t.dot(self.Wix)+ h_t1.dot(self.Wih) + self.bih)
            Cdasht = T.tanh(x_t.dot(self.WCx)+ h_t1.dot(self.WCh) + self.bCh)
            ot = T.nnet.sigmoid(x_t.dot(self.Wox)+ h_t1.dot(self.Woh) + self.boh)
            C_t = ft*C_t1 + it*Cdasht
            ot = T.nnet.sigmoid(x_t.dot(self.Wox)+ h_t1.dot(self.Woh) + self.boh)
            h_t = ot*T.tanh(C_t)
            return h_t,C_t
        
        #forward run of sequence
        [h,C],_ = theano.scan(
                                fn = recurrance,
                                sequences=Ei,
                                outputs_info=[self.h0,self.C0],
                                n_steps=Ei.shape[0]
                            )
        
        py_x = T.nnet.softmax(h.dot(self.Wo) + self.bo)
        
        prediction = T.argmax(py_x,axis=1)
        
        cost = -1*T.mean(T.log(py_x[T.arange(thY.shape[0]),thY]))
        grads = T.grad(cost,self.params)
        
        
        updates = [(p,p + (mu*dp - alpha*g)) for g,p,dp in zip(grads,self.params,self.dparams)] + [
            (dp,mu*dp - alpha*g) for g,p,dp in zip(grads,self.params,self.dparams)
        ]
        self.train = theano.function(
            inputs = [thX,thY],
            outputs = [cost,prediction],
            updates = updates
        )
        
        self.predict = theano.function(
            inputs = [thX],
            outputs = [prediction]
        )
        
        count = 0
        costs = []
        n_total = sum([len(row) for row in X])
        for e in xrange(epochs):
            first = True
            cost = 0
            total_correct_words = 0
            X = shuffle(X)
            for row in X:
                trainX = [0] + row
                trainY = row + [1]
                [c,prediction] = self.train(thX=trainX,thY=trainY)
                for p,t in zip(prediction,trainY):
                    if p==t:
                        total_correct_words = total_correct_words + 1
                count = count + 1
                cost = cost + c
            print "Cost is",cost," Prediction is",(1.0*total_correct_words)/n_total
            costs.append(cost)
        plt.plot(costs)
    def generate(self,word2idx,num_lines,sentences):
        
            current = 0

            #creating the inverted index map
            idx2word = {}
            for key,value in word2idx.iteritems():
                idx2word[value]=key

            #start_words = np.random.choice(len(sentences),num_lines)
            X = [sentences[np.random.choice(len(sentences))][0]]
            while current<num_lines:
                #print X
                [pred] = self.predict(thX=X)
                #print pred
                next_word = pred[-1]
                if idx2word[next_word]=='END':
                    current = current + 1
                    #print "\n"
                    sentence = " "
                    for num in X:
                        sentence += " " + idx2word[num] 
                    print sentence
                    X = [sentences[np.random.choice(len(sentences))][0]]                
                    #if current<num_lines:
                    #    print idx2word[X[0]]
                else:
                    #print idx2word[next_word]
                    X.append(next_word)
    def save(self,filename):
            np.savez(filename, *[param.get_value() for param in self.params])
            npz = np.load(filename)
            print npz['arr_0']
    @staticmethod
    def load(filename):
            if not os.path.exists(filename):
                raise Excpetion("File"+path+"does not exist")
            params = ["We","C0","h0","Wfx","Wfh","bfh","Wix","Wih","bih",
                      "WCx","WCh","bCh","Wox","Woh","boh","Wo","bo"]
            count = 0
            npz = np.load(filename)
            We = npz['arr_0']
            C0 = npz['arr_1']
            M = C0.shape[0]
            D,V = We.shape
            
            ls = lstm(M,D,V)
            
            for p in params:
                setattr(ls,p,theano.shared(npz["arr_"+`count`]))
                count += 1
            ls.set_variables()
            return ls

In [102]:
ls = lstm(30,30,len(vocabulary))

In [110]:
ls.fit(sentences,epochs=20)
ls.save("lstm_30_30_20.npz")

Cost is 11019.9409834  Prediction is 0.132512131392
Cost is 10603.9525709  Prediction is 0.134005225831
Cost is 10217.7603248  Prediction is 0.134005225831
Cost is 9962.96698169  Prediction is 0.134005225831
Cost is 9791.45621623  Prediction is 0.134005225831
Cost is 9589.93050361  Prediction is 0.134005225831
Cost is 9394.50426392  Prediction is 0.134005225831
Cost is 9233.90379919  Prediction is 0.134005225831
Cost is 9108.08656161  Prediction is 0.134005225831
Cost is 9005.27826983  Prediction is 0.134005225831
Cost is 8926.76153202  Prediction is 0.134005225831
Cost is 8866.32549539  Prediction is 0.134005225831
Cost is 8814.31202894  Prediction is 0.134005225831
Cost is 8762.95108485  Prediction is 0.134191862635
Cost is 8709.99035728  Prediction is 0.143243747667
Cost is 8669.30775309  Prediction is 0.15313549832


MemoryError: alloc failed
Apply node that caused the error: Alloc(TensorConstant{(1L, 1L) of 0.0}, Shape_i{0}.0, Shape_i{1}.0)
Toposort index: 70
Inputs types: [TensorType(float64, (True, True)), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(1L, 1L), (), ()]
Inputs strides: [(8L, 8L), (), ()]
Inputs values: [array([[ 0.]]), array(2446L, dtype=int64), array(30L, dtype=int64)]
Outputs clients: [[AdvancedIncSubtensor1{inplace,inc}(Alloc.0, IncSubtensor{InplaceInc;int64::}.0, thX)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "C:\Users\parmeets\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):
  File "C:\Users\parmeets\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-110-38aa852

In [106]:
ls.generate(vocabulary,10,sentences)

  I
  How
  The
  Walls
  So
  Dont
  Was
  With
  The
  To


In [107]:
ls2 = lstm.load("lstm_30_30_20.npz")
ls2.generate(vocabulary,10,sentences)


  On
  But
  They
  And
  Except
  Havent
  And
  And
  Our
  Two


In [108]:
ls2 = lstm.load("lstm_30_30.npz")

In [109]:
ls2.generate(vocabulary,10,sentences)

  I
  Like
  You
  Legitimately
  Its
  Moving
  There
  From
  I
  My


In [21]:
npfile = np.load("save.npz")